In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import pandas as pd
import spacy
from spacy.util import minibatch, compounding
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
import numpy as np
from numpy import array, unique, array_equal
max_fatures = 1000


def load_data():
    print("Loading Data...")
    PrimaryEmotion = pd.read_csv('emotion.xls.csv')
    print("Data Loaded")
    return (PrimaryEmotion['sentence'], array(PrimaryEmotion['emotion']))


#Below the data is sorted into nine emotion groups. Eight of the groups are the outter layer of the wheel, or the combinations of 
#two emotion groups. The ninth group is "Ambiguous" and "Neutral" put together.
def sort_to_9_emotions(sentence_list, emotion_list):
    sorted_list = []
    sorted_emo = []
    for (data, emo) in zip(sentence_list, emotion_list):
        if (emo == 'Contempt' or emo == 'Anger' or emo == 'Disgust'):
            sorted_list.append(data)
            sorted_emo.append("Contempt")
        if (emo == 'Aggression' or emo == 'Anticipation' or emo == 'Anger'):
            sorted_list.append(data)
            sorted_emo.append("Aggression")
        if (emo == 'Optimism' or emo == 'Anticipation' or emo == 'Joy'):
            sorted_list.append(data)
            sorted_emo.append("Optimism")
        if (emo == 'Love' or emo == 'Joy' or emo == 'Trust'):
            sorted_list.append(data)
            sorted_emo.append("Love")
        if (emo == 'Submission' or emo == 'Trust' or emo == 'Fear'):
            sorted_list.append(data)
            sorted_emo.append("Submission")
        if (emo == 'Awe' or emo == 'Fear' or emo == 'Surprise'):
            sorted_list.append(data)
            sorted_emo.append("Awe")
        if (emo == 'Disapproval' or emo == 'Surprise' or emo == 'Saddness'):
            sorted_list.append(data)
            sorted_emo.append("Disapproval")
        if (emo == 'Remorse' or emo == 'Saddness' or emo == 'Disgust'):
            sorted_list.append(data)
            sorted_emo.append("Remorse")
        if (emo == 'Neutral' or emo == 'Ambiguous'):
            sorted_list.append(data)
            sorted_emo.append("Neutral")
    return (sorted_list, sorted_emo)



def tokenize(sentences):
    print("Tokenizing...")
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(sentences)
    X = tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(X)
    print("Tokenization Completed")
    return X



def train(tokened_sentences, emotion_list, weigh_even=True):
    print("Training...")
    embed_dim = 128
    lstm_out = 196
    test_percent=.2

    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = tokened_sentences.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    
    # len(set(emotion_list)) is a hacky way of geting the number of unique elements
    # in a regualar python list (non-numpy)
    model.add(Dense(unique(emotion_list).size,activation='softmax'))

    
    model.compile(loss = 'categorical_crossentropy', optimizer='Adam',metrics = ['accuracy'])
    #print(model.summary())
    
    Y = pd.get_dummies(emotion_list).values
    X_train, X_test, Y_train, Y_test = train_test_split(tokened_sentences,Y, test_size = test_percent, random_state = 42)

    

    batch_size = 32
    if (weigh_even):
        y_integers = np.argmax(Y_train, axis=1)
        class_weights = class_weight.compute_sample_weight('balanced', unique(emotion_list), y_integers)
    
    model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1)
    print("Training Completed")
    print("Testing Against Control... (% of the data) ", test_percent)
    score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
    print("Score   :", score)
    print("Accuracy:", acc)
    return (model, X_test)

### When Splitting data (train_test_split), we don't retain where in the 
### origional set the data is located, thus it takes
### a little trickery to see the results while while only testing against untrained data
### top_predictions variable changes how many predictions given
def test(model, X_test, tokenized_data, sentence_list, emotion_list, top_predictions=3):
    predictions = model.predict(tokenized_data, batch_size=32)
    error = 0
    for i in range(len(predictions)):
     
        #  "If the current tokenized data array is in X_test (untrained tokenized arrays)
        
        if(any(array_equal(tokenized_data[i], x) for x in X_test)):
            print("\n\n")

            
            pos = list(predictions[i]).index(max(predictions[i]))
            if (unique(emotion_list)[pos] != emotion_list[i]):
                error += 1
            temp = predictions[i]
            
            print(sentence_list[i])

            for j in range(top_predictions):
                pos = list(temp).index(max(temp))
                print("\n # {%s} Predicted emotion : ",j+1, unique(emotion_list)[pos])
                temp[pos] = 0
            print("Actual emotion   : ", emotion_list[i])

    print("%  Accuracy when against untrained set: ", 1- (float(error) / len(X_test)))

#################################
#############MAIN################
#################################
#   jupyter notebook is weird   #




(sentence_list, emotion_list) = load_data()


### Comment the Below line for all 18 emotions. This sorts into "positive" and "negative"
sentence_list, emotion_list = sort_to_9_emotions(sentence_list, emotion_list)

tokenized_data = tokenize(sentence_list)

(model, X_test) = train(tokenized_data, emotion_list)

test(model, X_test, tokenized_data, sentence_list, emotion_list)


Loading Data...
Data Loaded
Tokenizing...
Tokenization Completed
Training...
Epoch 1/10
2592/2592 [==============================] - 13s 5ms/step - loss: 2.1033 - acc: 0.1941
Epoch 2/10
2592/2592 [==============================] - 11s 4ms/step - loss: 2.0503 - acc: 0.2458
Epoch 3/10
2592/2592 [==============================] - 12s 4ms/step - loss: 1.9246 - acc: 0.3013
Epoch 4/10
2592/2592 [==============================] - 11s 4ms/step - loss: 1.7437 - acc: 0.3546
Epoch 5/10
2592/2592 [==============================] - 10s 4ms/step - loss: 1.5879 - acc: 0.4101
Epoch 6/10
2592/2592 [==============================] - 10s 4ms/step - loss: 1.4760 - acc: 0.4356
Epoch 7/10
2592/2592 [==============================] - 11s 4ms/step - loss: 1.3980 - acc: 0.4657
Epoch 8/10
2592/2592 [==============================] - 11s 4ms/step - loss: 1.3286 - acc: 0.4834
Epoch 9/10
2592/2592 [==============================] - 11s 4ms/step - loss: 1.2335 - acc: 0.5112
Epoch 10/10
2592/2592 [==================


 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Remorse
Actual emotion   :  Contempt



Surrealism was definitely the last straw.

 # {%s} Predicted emotion :  1 Contempt

 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Remorse
Actual emotion   :  Remorse



Yanto waited for a reply to his offer with bated breath.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Neutral

 # {%s} Predicted emotion :  3 Aggression
Actual emotion   :  Aggression



Yanto waited for a reply to his offer with bated breath.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Neutral

 # {%s} Predicted emotion :  3 Aggression
Actual emotion   :  Optimism



The slightest skirl of the pipes makes the retired cleaner go weak at the knees, break out in a cold sweat and shake uncontrollably.

 # {%s} Predicted emotion :  1 Aggression

 # {%s} Predicted emotion :  2 Optimism

 # {%s} Predicted emotion :  3 Neutral
Act

The reason why I have a soft spot for this notebook, he wrote.

 # {%s} Predicted emotion :  1 Love

 # {%s} Predicted emotion :  2 Optimism

 # {%s} Predicted emotion :  3 Aggression
Actual emotion   :  Love



But her silence, and her eyebrows, speak volumes.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Love

 # {%s} Predicted emotion :  3 Aggression
Actual emotion   :  Neutral



He won't eat dumplings, if he sees dumplings in a stew he'll puke.

 # {%s} Predicted emotion :  1 Contempt

 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Remorse
Actual emotion   :  Contempt



He won't eat dumplings, if he sees dumplings in a stew he'll puke.

 # {%s} Predicted emotion :  1 Contempt

 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Remorse
Actual emotion   :  Remorse



Providing the income you will need after you stop work can take a load off your mind.

 # {%s} Predicted emotion :  1 Aggression

 # {%s} Pre




Erm since we've got together we, we are on top of the world.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Love

 # {%s} Predicted emotion :  3 Neutral
Actual emotion   :  Optimism



Erm since we've got together we, we are on top of the world.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Love

 # {%s} Predicted emotion :  3 Neutral
Actual emotion   :  Love



I will go bananas.

 # {%s} Predicted emotion :  1 Aggression

 # {%s} Predicted emotion :  2 Contempt

 # {%s} Predicted emotion :  3 Optimism
Actual emotion   :  Contempt



I will go bananas.

 # {%s} Predicted emotion :  1 Aggression

 # {%s} Predicted emotion :  2 Contempt

 # {%s} Predicted emotion :  3 Optimism
Actual emotion   :  Aggression



It was your harebrained idea.

 # {%s} Predicted emotion :  1 Disapproval

 # {%s} Predicted emotion :  2 Awe

 # {%s} Predicted emotion :  3 Neutral
Actual emotion   :  Disapproval



It may well be necessary to botch up rep




It was all downhill to the causeway, and the turf was smooth.

 # {%s} Predicted emotion :  1 Neutral

 # {%s} Predicted emotion :  2 Optimism

 # {%s} Predicted emotion :  3 Love
Actual emotion   :  Neutral



I'm gon na fall flat on my bum.

 # {%s} Predicted emotion :  1 Aggression

 # {%s} Predicted emotion :  2 Optimism

 # {%s} Predicted emotion :  3 Contempt
Actual emotion   :  Submission



I'm gon na fall flat on my bum.

 # {%s} Predicted emotion :  1 Aggression

 # {%s} Predicted emotion :  2 Optimism

 # {%s} Predicted emotion :  3 Contempt
Actual emotion   :  Awe



It might be a good one and if you could get an appointment as their vet it would be another feather in your cap

 # {%s} Predicted emotion :  1 Neutral

 # {%s} Predicted emotion :  2 Disapproval

 # {%s} Predicted emotion :  3 Optimism
Actual emotion   :  Optimism



It was a test that the Three-Ninetieth passed with flying colours destroying thirteen enemy aircraft and achieving excellent bombing results.


...

Boxing and Las Vegas, joined in very unholy matrimony, are waiting with bated breath and heavily baited betting slips for the outcome of a fight that can be almost relied upon to restore some credibility to the heavyweight division.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Neutral
Actual emotion   :  Aggression



Boxing and Las Vegas, joined in very unholy matrimony, are waiting with bated breath and heavily baited betting slips for the outcome of a fight that can be almost relied upon to restore some credibility to the heavyweight division.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Neutral
Actual emotion   :  Optimism



Sparks fly along the nerve wires, pins and needles and it's over.

 # {%s} Predicted emotion :  1 Awe

 # {%s} Predicted emotion :  2 Disapproval

 # {%s} Predicted emotion :  3 Submission
Actual emotion   :  Neutral








Lo and behold, the thief was caught red-handed.

 # {%s} Predicted emotion :  1 Disapproval

 # {%s} Predicted emotion :  2 Neutral

 # {%s} Predicted emotion :  3 Awe
Actual emotion   :  Optimism



Lo and behold, the thief was caught red-handed.

 # {%s} Predicted emotion :  1 Disapproval

 # {%s} Predicted emotion :  2 Neutral

 # {%s} Predicted emotion :  3 Awe
Actual emotion   :  Love



By morning, if anyone troubled to come looking for her, they would probably find her poor, frozen body rooted to the spot by a cage of spindly icicles.

 # {%s} Predicted emotion :  1 Contempt

 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Remorse
Actual emotion   :  Contempt



By morning, if anyone troubled to come looking for her, they would probably find her poor, frozen body rooted to the spot by a cage of spindly icicles.

 # {%s} Predicted emotion :  1 Contempt

 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Remorse
Actual emotion   

The performance went off without a hitch and the Vancouver Symphony Orchestra, with Dr Otto Klemperer conducting, played brilliantly.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Love

 # {%s} Predicted emotion :  3 Neutral
Actual emotion   :  Love



He left them in safe hands.

 # {%s} Predicted emotion :  1 Submission

 # {%s} Predicted emotion :  2 Awe

 # {%s} Predicted emotion :  3 Love
Actual emotion   :  Love



He left them in safe hands.

 # {%s} Predicted emotion :  1 Submission

 # {%s} Predicted emotion :  2 Awe

 # {%s} Predicted emotion :  3 Love
Actual emotion   :  Submission



Thirdly, there is therefore never a single hypothesis at stake.

 # {%s} Predicted emotion :  1 Neutral

 # {%s} Predicted emotion :  2 Submission

 # {%s} Predicted emotion :  3 Optimism
Actual emotion   :  Neutral



The sign outside may say Cad Mle Fate, but inside you watch your step.

 # {%s} Predicted emotion :  1 Aggression

 # {%s} Predicted emotion :  2 Optim




Members wishing to express an opinion in the debate later this afternoon, any suggestion that the additional two hours provided is adequate is clearly wide of the mark?

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Disapproval
Actual emotion   :  Disapproval



We were trying to diverse the music with the mixture of soul and reggae, though it wasn't really the done thing at that point.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Neutral
Actual emotion   :  Aggression



We were trying to diverse the music with the mixture of soul and reggae, though it wasn't really the done thing at that point.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Aggression

 # {%s} Predicted emotion :  3 Neutral
Actual emotion   :  Optimism



Venus fly trap, who've just released their latest' Achilles Heel' single on Danceteria.

 # {%s} 




Vicky is thrilled to bits that the dog has been found.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Love

 # {%s} Predicted emotion :  3 Aggression
Actual emotion   :  Optimism



Vicky is thrilled to bits that the dog has been found.

 # {%s} Predicted emotion :  1 Optimism

 # {%s} Predicted emotion :  2 Love

 # {%s} Predicted emotion :  3 Aggression
Actual emotion   :  Love



Come and take potluck, eat humble pie whatever.

 # {%s} Predicted emotion :  1 Submission

 # {%s} Predicted emotion :  2 Neutral

 # {%s} Predicted emotion :  3 Awe
Actual emotion   :  Neutral



Many centuries of trial and error would elapse before Ireland's dark secret would be revealed.

 # {%s} Predicted emotion :  1 Disapproval

 # {%s} Predicted emotion :  2 Optimism

 # {%s} Predicted emotion :  3 Awe
Actual emotion   :  Awe



Many centuries of trial and error would elapse before Ireland's dark secret would be revealed.

 # {%s} Predicted emotion :  1 Disapproval

 # {